In [19]:
import pandas as pd
import openpyxl
import time

In [20]:
path_file = 'data/СВОД_4 кв_2023.xlsx'

In [21]:
# получаем список листов
temp_wb = openpyxl.load_workbook(path_file,read_only=True)
lst_file = temp_wb.sheetnames
temp_wb.close()

In [22]:
lst_file

['Свод_2019', 'Свод_2020', 'Свод_2021', 'Свод_2022', 'свод_2023']

In [23]:
temp_reg_df = pd.read_excel(path_file,sheet_name='Свод_2019',nrows=1)

In [25]:
# Получаем регионы на листе
temp_lst_regions = [region for region in temp_reg_df.columns if 'Unnamed' not in region]

In [26]:
temp_lst_regions

['Амурская область',
 'Астраханская область',
 'Волгоградская область',
 'Воронежская область',
 'Иркутская область',
 'Кемеровская область',
 'Московская область',
 'Новгородская область',
 'РСО-Алания',
 'Республика Татарстан',
 'Свердловская область',
 'Тверская область',
 'Томская область',
 'Тюменская область']

In [27]:
temp_df = pd.read_excel(path_file,sheet_name='Свод_2019',skiprows=1)

In [29]:
temp_df.head(1)

,№ показателя,Наименование индикатора/показателя,№ строки,Единица измерения,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!),"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)",Фактическое значение показателя нарастающим итогом с года создания ЦОПП,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .1,...,Фактическое значение показателя нарастающим итогом с года создания ЦОПП .10,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).11,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .11,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .12,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .13
0,1,2,3.0,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# удаляем лишние колонки
temp_df.drop(columns=['№ показателя','№ строки','Единица измерения'],inplace=True)

In [32]:
temp_df.head(5)

,Наименование индикатора/показателя,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!),"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)",Фактическое значение показателя нарастающим итогом с года создания ЦОПП,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .1,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).2,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).2",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .2,...,Фактическое значение показателя нарастающим итогом с года создания ЦОПП .10,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).11,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .11,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .12,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .13
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Созданы цифровая платформа и сайт ЦОПП в сети ...,NaN,NaN,NaN,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,NaN,NaN,NaN,...,https://copp66.ru/cifrovaya_platforma\nhttps:/...,создана в 2019 г.,https://copp69.ru/,https://copp69.ru/,https://copp70.ru/,https://copp70.ru/,https://copp70.ru/,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...
3,"Количество обращений, зарегистрированных на ци...",NaN,NaN,NaN,57,252,309,NaN,NaN,NaN,...,9423,679,3860,5477,3633,9167,41027,2870,6003,18693
4,"Численность граждан Российской Федерации, охва...",NaN,NaN,NaN,8651,20046,69707,NaN,NaN,NaN,...,200034,10012,20560,53000,17196,24061,83437,32677,96523,243567


In [33]:
# убираем лишние строки с значением 1 или 2
temp_df = temp_df[~temp_df['Наименование индикатора/показателя'].isin([1,2])]

In [34]:
temp_df.head()

,Наименование индикатора/показателя,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!),"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)",Фактическое значение показателя нарастающим итогом с года создания ЦОПП,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).1",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .1,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).2,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).2",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .2,...,Фактическое значение показателя нарастающим итогом с года создания ЦОПП .10,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).11,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).11",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .11,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).12,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).12",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .12,Фактическое значение по итогам IV квартала 2023 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов).13",Фактическое значение показателя нарастающим итогом с года создания ЦОПП .13
2,Созданы цифровая платформа и сайт ЦОПП в сети ...,NaN,NaN,NaN,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,https://copp.edu.astrobl.ru/,NaN,NaN,NaN,...,https://copp66.ru/cifrovaya_platforma\nhttps:/...,создана в 2019 г.,https://copp69.ru/,https://copp69.ru/,https://copp70.ru/,https://copp70.ru/,https://copp70.ru/,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...,https://platform.copp72.ru/ https://www.copp72...
3,"Количество обращений, зарегистрированных на ци...",NaN,NaN,NaN,57,252,309,NaN,NaN,NaN,...,9423,679,3860,5477,3633,9167,41027,2870,6003,18693
4,"Численность граждан Российской Федерации, охва...",NaN,NaN,NaN,8651,20046,69707,NaN,NaN,NaN,...,200034,10012,20560,53000,17196,24061,83437,32677,96523,243567
5,из них (из п.3) в возрасте:\n– до 16 лет,NaN,NaN,NaN,7344,13943,26388,NaN,NaN,NaN,...,101980,2164,6237,33948,309,1377,12300,22295,68121,189952
6,– от 16 до 21 года,NaN,NaN,NaN,689,3798,16253,NaN,NaN,NaN,...,57014,7172,11264,14726,14804,17455,37909,3102,13164,25194
